In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures
import warnings
warnings.filterwarnings("ignore")
np.random.seed(42)

In [2]:
df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", sep = ';')
y = df.pop('quality')

In [3]:
for i in df.columns:
    df[i] = df[i].fillna(np.mean(df[i]))
train, test, y_train, y_test = train_test_split(df, y, test_size = 0.2)

In [4]:
lr = LogisticRegression()
lr.fit(train, y_train)
y_pred = lr.predict(test)
print('Accuracy score baseline:', accuracy_score(y_test, y_pred))

Accuracy score baseline: 0.5153061224489796


In [8]:
def fit_predict(train, test, y_train, y_test, scaler, 
                n_neighbours, metric = 'manhattan', weights = 'uniform'):
    train_scaled = scaler.fit_transform(train)
    test_scaled = scaler.transform(test)        
    knn = KNeighborsClassifier(n_neighbors=n_neighbours, metric=metric, 
                               weights=weights, n_jobs = 4)
    knn.fit(train_scaled, y_train)
    y_pred = knn.predict(test_scaled)
    print(accuracy_score(y_test, y_pred))

### Neighbours tuning

In [13]:
for k in range(1,12):
    print('Accuracy score on kNN using n_neighbours = {0}:'.format(2**k), end = ' ')
    fit_predict(train, test, y_train, y_test, StandardScaler(), 2**k)

Accuracy score on kNN using n_neighbours = 2: 0.5724489795918367
Accuracy score on kNN using n_neighbours = 4: 0.5551020408163265
Accuracy score on kNN using n_neighbours = 8: 0.5438775510204081
Accuracy score on kNN using n_neighbours = 16: 0.5418367346938775
Accuracy score on kNN using n_neighbours = 32: 0.5520408163265306
Accuracy score on kNN using n_neighbours = 64: 0.5387755102040817
Accuracy score on kNN using n_neighbours = 128: 0.5295918367346939
Accuracy score on kNN using n_neighbours = 256: 0.5163265306122449
Accuracy score on kNN using n_neighbours = 512: 0.5040816326530613
Accuracy score on kNN using n_neighbours = 1024: 0.47244897959183674
Accuracy score on kNN using n_neighbours = 2048: 0.44081632653061226


In [7]:
for k in np.logspace(2, 11, base = 2, num = 11, dtype=int).tolist():
    print('Accuracy score on kNN using n_neighbours = {0}:'.format(k), end = ' ')
    fit_predict(train, test, y_train, y_test, StandardScaler(), k)

Accuracy score on kNN using n_neighbours = 4: 0.5551020408163265
Accuracy score on kNN using n_neighbours = 7: 0.5581632653061225
Accuracy score on kNN using n_neighbours = 13: 0.5438775510204081
Accuracy score on kNN using n_neighbours = 25: 0.55
Accuracy score on kNN using n_neighbours = 48: 0.5469387755102041
Accuracy score on kNN using n_neighbours = 90: 0.5346938775510204
Accuracy score on kNN using n_neighbours = 168: 0.5285714285714286
Accuracy score on kNN using n_neighbours = 315: 0.5153061224489796
Accuracy score on kNN using n_neighbours = 588: 0.4928571428571429
Accuracy score on kNN using n_neighbours = 1097: 0.4673469387755102
Accuracy score on kNN using n_neighbours = 2048: 0.44081632653061226


### Metric tuning

In [14]:
for metric in ['euclidean', 'cosine', 'manhattan', 'chebyshev']:
    print('Accuracy score on kNN using {} metric and {} neighbours:'.format(metric,k), end = ' ')
    fit_predict(train, test, y_train, y_test, StandardScaler(), 2, metric)

Accuracy score on kNN using euclidean metric and 11 neighbours: 0.573469387755102
Accuracy score on kNN using cosine metric and 11 neighbours: 

Process ForkPoolWorker-23:
Process ForkPoolWorker-24:
Process ForkPoolWorker-22:
Process ForkPoolWorker-21:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self

KeyboardInterrupt: 

### Weighted kNN

In [9]:
for weights in ['uniform', 'distance']:
    print('Accuracy score on kNN using weights = {0}:'.format(weights), end = ' ')
    fit_predict(train, test, y_train, y_test, StandardScaler(), 2, 'chebyshev', weights = weights)

Accuracy score on kNN using weights = uniform: 0.5744897959183674
Accuracy score on kNN using weights = distance: 0.6489795918367347


### Engineering

In [10]:
def create_poly(train,test,degree):
    poly = PolynomialFeatures(degree=degree)
    train_poly = poly.fit_transform(train)
    test_poly = poly.fit_transform(test)
    return train_poly,test_poly

In [11]:
for degree in [1,2,3]:
    train_poly, test_poly = create_poly(train, test, degree)
    print('Polynomial degree',degree)
    fit_predict(train_poly, test_poly, y_train, y_test, StandardScaler(), 2, 'chebyshev', weights = 'distance')
    print(10*'-')
    
train_poly, test_poly = create_poly(train, test, 2) 

Polynomial degree 1
0.6489795918367347
----------
Polynomial degree 2
0.6408163265306123
----------
Polynomial degree 3
0.6428571428571429
----------


In [12]:
def feat_eng(df):
    df['eng1'] = df['fixed acidity'] * df['pH']
    df['eng2'] = df['total sulfur dioxide'] / df['free sulfur dioxide']
    df['eng3'] = df['sulphates'] / df['chlorides']
    df['eng4'] = df['chlorides'] / df['sulphates']
    return df

train = feat_eng(train)
test = feat_eng(test)

In [13]:
print('Accuracy score after engineering:', end = ' ')
fit_predict(train, test, y_train, y_test, StandardScaler(), 2, 'chebyshev', weights = 'distance')

Accuracy score after engineering: 0.6704081632653062


In [14]:
original_score = 0.514285714286
best_score = 0.670408163265
improvement = np.abs(np.round(100*(original_score - best_score)/original_score,2))
print('overall improvement is {} %'.format(improvement))

overall improvement is 30.36 %
